In [ ]:
import os
import datetime
import numpy as np 
import pandas as pd
import arcpy
from arcpy import env

In [ ]:
def get_date():
    return datetime.datetime.now().strftime('%Y%m%d')
def csv_to_raster(csv_path, reference_raster_path):
    # Read CSV data
    data = pd.read_csv(csv_path, index_col=False)
    # Convert to float
    data = data.astype('float32')
    # Merge all columns except the first one
    merged_data = pd.concat([data[col] for col in data.columns], axis=0, ignore_index=True)
    merged_data.dropna(inplace=True)
    
    # Get reference raster properties
    dem_array = arcpy.RasterToNumPyArray(reference_raster_path)
    # check shape size
    print("stander shape:",dem_array.shape)
    print("csv_shpe",merged_data.shape)
    # Reshape data to match reference raster
    raster_array = np.reshape(merged_data.values, dem_array.shape)
    
    # return raster_array
    return raster_array
# 掩膜提取
def mask_raster(array,mask_ele,cell_size):
    out_raster = arcpy.NumPyArrayToRaster(
    array,
    arcpy.Point(arcpy.env.extent.XMin, arcpy.env.extent.YMin),
    cell_size,
    cell_size,
)
    """按掩膜提取栅格,空间参考设定为:CGCS2000_3_Degree_GK_CM_108E"""
    output_coordinate_system = arcpy.Describe(mask_ele).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=output_coordinate_system,snapRaster=mask_ele, cellSize=mask_ele):
        result_raster = arcpy.sa.ExtractByMask(out_raster, mask_ele, "INSIDE")
        return result_raster

In [ ]:
stander_path = r"D:\ArcGISProjects\workspace\shbyq\feature_raster_file\features_data_dy.gdb\DEM"
out_path = r"D:\ArcGISProjects\workspace\shbyq\feature_soilproperty_result\feature_soilproperty_dy.gdb"
csv_root_path = r'F:\cache_data\pre_property_table\dy\rfrk_pre'

In [ ]:
env.workspace = out_path
env.extent = stander_path

In [ ]:
# ---
for one_csv in os.listdir(csv_root_path):
    temp_csv_path = os.path.join(csv_root_path,one_csv)
    print(temp_csv_path)
    out_name = f"{one_csv.split('.')[0]}_{get_date()}"
    # 执行转换
    temp_raster_array = csv_to_raster(temp_csv_path,stander_path)
    print(temp_raster_array.dtype)
    # 写入栅格
    result_raster = mask_raster(temp_raster_array,stander_path, 5)
    result_raster.save(out_name)
    print(f"{one_csv} 转换完成")